In [1]:
import requests
import json
from IPython.display import Markdown

In [5]:
def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt, "history": []}
    response = requests.post(url='http://127.0.0.1:8000', headers=headers, data=json.dumps(data))
    return response.json()['response']

get_completion("你好")

'你好👋!我是人工智能助手 ChatGLM-6B,很高兴见到你,欢迎问我任何问题。'

# 两个基本原则

## 原则一：编写清晰、具体的指令

&emsp;&emsp; 你应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并减少你得到无关或不正确响应的可能。编写清晰的指令不意味着简短的指令，因为在许多情况下，更长的提示实际上更清晰且提供了更多上下文，这实际上可能导致更详细更相关的输出。

&emsp;&emsp;**策略一：使用分隔符清晰地表示输入的不同部分，分隔符可以是：```，""，<>，<tag>，<\tag>等 ```**

&emsp;&emsp;你可以使用任何明显的标点符号将特定的文本部分与提示的其余部分分开。这可以是任何可以使模型明确知道这是一个单独部分的标记。使用分隔符是一种可以避免提示注入的有用技术。提示注入是指如果用户将某些输入添加到提示中，则可能会向模型提供与您想要执行的操作相冲突的指令，从而使其遵循冲突的指令而不是执行您想要的操作。即，输入里面可能包含其他指令，会覆盖掉你的指令。对此，使用分隔符是一个不错的策略。

&emsp;&emsp;以下是一个例子，我们给出一段话并要求 GPT 进行总结，在该示例中我们使用 ``` 来作为分隔符

In [3]:
text = f"""
你应该提供尽可能清晰、具体的指示，以表达你希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示与写简短的提示混淆。\
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰、具体的指示,让模型执行所需的任务,减少无关或不正确响应的可能性。长提示可以提供更多上下文信息,导致更详细和相关的输出。


&emsp;&emsp;**策略二：要求一个结构化的输出，可以是 Json、HTML 等格式**

&emsp;&emsp;第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析，例如，你可以在 Python 中将其读入字典或列表中。。

&emsp;&emsp;在以下示例中，我们要求 GPT 生成三本书的标题、作者和类别，并要求 GPT 以 Json 的格式返回给我们，为便于解析，我们指定了 Json 的键。

In [4]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，\
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

```json
{
  "book_id": 1,
  "title": "The Matrix",
  "author": "Zack Snyder",
  "genre": "Sci-Fi"
},
{
  "book_id": 2,
  "title": "To Kill a Mockingbird",
  "author": "Harper Lee",
  "genre": " Fiction"
},
{
  "book_id": 3,
  "title": "The Great Gatsby",
  "author": "F. Scott Fitzgerald",
  "genre": " Fiction"
}
```


&emsp;&emsp;**策略三：要求模型检查是否满足条件**

&emsp;&emsp;如果任务做出的假设不一定满足，我们可以告诉模型先检查这些假设，如果不满足，指示并停止执行。你还可以考虑潜在的边缘情况以及模型应该如何处理它们，以避免意外的错误或结果。

&emsp;&emsp;在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答未提供步骤。

In [8]:
# 有步骤的文本
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果你愿意，可以加一些糖或牛奶调味。\
就这样，你可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步: 把水烧开
第二步: 等待期间,拿一个杯子并把茶包放进去
第三步: 一旦水足够热,就把它倒在茶包上
第四步: 等待一会儿,让茶叶浸泡
第五步: 几分钟后,取出茶包
第六步: 如果你愿意,可以加一些糖或牛奶调味

未提供步骤


注：看得出来chatgl写出了六个步骤，当并没有给出最后一步**享受一杯美味的茶**，与chatgpt相比还有一点差距。

In [10]:
# 无步骤的文本
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
第一步: 去公园散步
第二步: 欣赏鲜花和树枝
第三步: 感受微风
第四步: 放松身心
第五步: 野餐或玩游戏
第六步: 感受大自然的美丽

未提供步骤


注：在官方教程的chatgpt回答**未提供步骤**，但chatglm依然给出了步骤的总结，最后甚至还写了**未提供步骤**

&emsp;&emsp;**策略四：提供少量示例**

&emsp;&emsp;即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

&emsp;&emsp;例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和一个祖父之间的对话的例子。孩子说，“教我耐心”，祖父用这些隐喻回答。因此，由于我们已经告诉模型要以一致的语气回答，现在我们说“教我韧性”，由于模型已经有了这个少样本示例，它将以类似的语气回答下一个任务。

In [11]:
prompt = f"""
你的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

韧性是一种能够承受压力和克服困难的品质。以下是一些培养韧性的方法:

1. 坚持锻炼:身体锻炼有助于增强韧性,包括身体的耐力和灵活性。

2. 学会放松:压力和焦虑可能会影响韧性,因此学会放松身心可以帮助缓解紧张情绪和增强韧性。

3. 学习解决问题:解决问题需要耐心和灵活性,学习如何分析问题并提供有效的解决方案可以提高韧性。

4. 坚持学习:不断学习新知识可以帮助提高自己的能力和应对不同的情况。

5. 培养同理心:同理心可以帮助更好地理解他人的情绪和经历,从而提高韧性。

希望这些方法可以帮助培养韧性。


注：chatglm大失败

## 原则二：给模型时间去思考

&emsp;&emsp;如果模型匆忙地得出了错误的结论，您应该尝试重新构思查询，请求模型在提供最终答案之前进行一系列相关的推理。换句话说，如果您给模型一个在短时间或用少量文字无法完成的任务，它可能会猜测错误。这种情况对人来说也是一样的。如果您让某人在没有时间计算出答案的情况下完成复杂的数学问题，他们也可能会犯错误。因此，在这些情况下，您可以指示模型花更多时间思考问题，这意味着它在任务上花费了更多的计算资源。

**策略一：指定完成任务所需的步骤** 

&emsp;&emsp;接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果

&emsp;&emsp;首先我们描述了杰克和吉尔的故事，并给出一个指令。该指令是执行以下操作。首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和名称数。然后我们要用换行符分隔答案。

In [12]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1. 用一句话概括下面用三个反引号括起来的文本。

"兄妹杰克和吉尔去山顶井里打水,杰克绊了一块石头从山上滚了下来,吉尔紧随其后,但他们还是回到了温馨的家中,尽管出了这样的意外,他们的冒险精神依然没有减弱,继续充满愉悦地探索。"

2. 将摘要翻译成法语。

"J'ai vu une ville de 100 000 habitants, en l'allusion à un tel village, qui est capable de changer la nature des choses. La plupart des villes ont une population de 100 000 habitants et laisse place à un certain nombre de points de vue différents, certains de ces points de vue incluent la nature de la地面, la taille de la maison, la présence d'un nom d'agent public, etc. C'est un tel village qui a réussi à changer la nature des choses, car c'est l'un de ces points de vue que je reflète. J'ai vu des différences notables de taille et de composition dans différents villes et je veux l'imaginer ici."

3. 在法语摘要中列出每个人名。

"J'ai vu un nom d'agent public, un nom de classe, un nom de groupe, un nom de village, un nom de ville, un nom de pays et un nom d'outre-tombe."

4. 输出一个 JSON 对象,其中包含以下键:French_summary,num_names。

"French_summary: J'ai vu un village de 10

注：寄的很失败，用一个更好的prompt

In [14]:
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
Translation:

In a charming village,兄妹杰克和吉尔 went to a山顶 well to drink water. They sang joyfully as they往上爬, but something unexpected happened -杰克 fell from the山上 and吉尔 followed, but they were not as受伤 as they expected. Despite the accident, their adventure spirit never decreased and they continued to explore with joy.

Name: 
在英语摘要中列出每个名称。

JSON:

```json
{
  "English_summary": "兄妹杰克和吉尔去山顶井里打水。他们一边唱着欢乐的歌,一边往上爬,然而不幸降临 - 杰克绊了一块石头,从山上滚了下来,吉尔紧随其后。虽然略有些摔伤,但他们还是回到了温馨的家中。尽管出了这样的意外,他们的冒险精神依然没有减弱,继续充满愉悦地探索。",
  "num_names": [
    "杰克",
    "吉尔"
  ]
}
```


注：可以看出姓名回答错误，但json的格式是对的，但回答格式也是错误的。总结：对了，但没完全对。

**策略二：指导模型在下结论之前找出一个自己的解法**

&emsp;&emsp;有时候，在明确指导模型在做决策之前要思考解决方案时，我们会得到更好的结果。

&emsp;&emsp;接下来我们会给出一个问题和一个学生的解答，要求模型判断解答是否正确

In [15]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。根据题目中提供的信息,我们可以列出以下三个费用:

1. 土地费用:100美元/平方英尺
2. 太阳能电池板费用:250美元/平方英尺
3. 维护费用:每年需要支付固定的10万美元,并额外支付每平方英尺10美元作为维护费用。

根据学生的解决方案,我们可以将这三个费用分别乘以发电站的大小x,得到首年运营的总费用:

1. 土地费用:100美元/平方英尺 × x
2. 太阳能电池板费用:250美元/平方英尺 × x
3. 维护费用:每年需要支付固定的10万美元,并额外支付每平方英尺10美元 × x

将三个费用相加,得到首年运营的总费用:

100美元/平方英尺 × x + 250美元/平方英尺 × x + 100,000美元 + 10美元/平方英尺 × x = 450x + 100,000美元

因此,首年运营的总费用为450x + 100,000美元。


注：**实际上学生的方案是不正确的，我们需要改进prompt**，chatgpt关于这个回答也是**学生的方案正确**，双方打个平手

In [17]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：

    首先，自己解决问题。
    然后将你的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

    问题：问题文本
    学生的解决方案：学生的解决方案文本
    实际解决方案和步骤：实际解决方案和步骤文本
    学生的解决方案和实际解决方案是否相同：是或否
    学生的成绩：正确或不正确

问题：

    我正在建造一个太阳能发电站，需要帮助计算财务。 
    - 土地费用为每平方英尺100美元
    - 我可以以每平方英尺250美元的价格购买太阳能电池板
    - 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

    设x为发电站的大小，单位为平方英尺。
    费用：
    1. 土地费用：100x
    2. 太阳能电池板费用：250x
    3. 维护费用：100,000+100x
    总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

实际解决方案和步骤与学生的解决方案相同。

根据学生的解决方案,我们可以列出每个费用的表达式:

1. 土地费用:100x
2. 太阳能电池板费用:250x
3. 维护费用:100,000+100x

然后,我们可以将这些费用相加,并将它们除以发电站大小的平方英尺数:

1. 土地费用:100x
2. 太阳能电池板费用:250x
3. 维护费用:100,000+100x
    总费用:100x+250x+100,000+100x=450x+100,000

最后,我们只需要将总费用除以发电站大小的平方英尺数,即可得到首年运营的总费用。因此,学生的解决方案和实际解决方案相同,都是:

450x+100,000/x=450x+100,000


注：维护费用算错了

# 局限性

**虚假知识**：模型偶尔会生成一些看似真实实则编造的知识

如果模型在训练过程中接触了大量的知识，它并没有完全记住所见的信息，因此它并不很清楚自己知识的边界。这意味着它可能会尝试回答有关晦涩主题的问题，并编造听起来合理但实际上并不正确的答案。我们称这些编造的想法为幻觉。

例如在如下示例中，我们要求告诉我们 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 产品的信息，事实上，这个公司是真实存在的，但产品是编造的，模型则会一本正经地告诉我们编造的知识。

In [18]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

Boie 公司生产的AeroGlide UltraSlim Smart Toothbrush是一款高端的电动牙刷,以下是它的一些相关信息:

1. 设计:AeroGlide UltraSlim Smart Toothbrush采用了最新的设计技术,具有非常简洁的设计,包括一个微小的柄和可定制的颜色选择。它还配备了智能感应技术,可以帮助用户更好地掌握清洁技巧。

2. 功能:AeroGlide UltraSlim Smart Toothbrush还配备了许多先进的功能,如智能感应、振动技术、声波技术等,可以更好地清洁牙齿。它还可以提供个性化的清洁方案,帮助用户更好地掌握清洁技巧。

3. 材料:AeroGlide UltraSlim Smart Toothbrush使用高品质的材料制作,包括不锈钢柄、电动牙刷头和刷毛等。这些材料可以更好地保持清洁效果,并提供更好的耐用性和舒适感。

4. 价格:AeroGlide UltraSlim Smart Toothbrush是一款高端的电动牙刷,价格相对较高。但是,它的质量和功能可以满足高端用户的需求,因此仍然受到许多消费者的青睐。

总的来说,AeroGlide UltraSlim Smart Toothbrush是一款功能强大、设计简洁、材料优质的高端电动牙刷,适合那些需要更好清洁效果和更个性化清洁方案的用户。


&emsp;&emsp;模型会输出看上去非常真实的编造知识，这有时会很危险。因此，请确保使用我们在本节中介绍的一些技巧，以尝试在构建自己的应用程序时避免这种情况。这是模型已知的一个弱点，也是我们正在积极努力解决的问题。在你希望模型根据文本生成答案的情况下，另一种减少幻觉的策略是先要求模型找到文本中的任何相关引用，然后要求它使用这些引用来回答问题，这种追溯源文档的方法通常对减少幻觉非常有帮助。

## 写在最后

本学习笔记通过DataWhale翻译编写的**吴恩达《ChatGPT Prompt Engineering for Developers》课程**, [课程地址](https://github.com/datawhalechina/prompt-engineering-for-developers.git)

但在笔记中全程使用chatglm的api接口，在一些内容上与chatgpt的回答作了比较。通过在趋动云部署chatglm-6b模型，调用api接口完成